## Accuracy: 100%
Things to note

1.	08/07/2023	Res num: 8,959,169 PNR is null in snow

In [1]:
from helpfile import *
import numpy as np

# Import and clean snowflake views

In [2]:
query = """select * from ANALYTICS_PROD.IOATAWARE.VW_PAYMENTS_RAW_DATA where "Payment Date"='08/02/2023'
    """
snowflake_pd_df = get_data(query)

In [3]:
# columns_to_remove = ['\tPercent of Full Leg','Leg Status','Flight Time','Sales Username']
# snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)

snowflake_pd_df['Payment Date'] = pd.to_datetime(snowflake_pd_df['Payment Date'])
#snowflake_pd_df.fillna('', inplace=True)
snowflake_pd_df.replace('', np.nan, inplace=True)
snowflake_pd_df['Exchange Rate'] = round(snowflake_pd_df['Exchange Rate'],6)
# snowflake_pd_df['Taxes'] = snowflake_pd_df['Taxes'].astype(float)
# snowflake_pd_df['Total Charge'] = snowflake_pd_df['Total Charge'].astype(float)
# snowflake_pd_df['Flight Nmbr'] = snowflake_pd_df['Flight Nmbr'].astype(int)

# Import and clean report data

In [4]:
report_df = pd.read_csv('Payment/Payments_Raw_Data_0802.csv', skiprows=0)   

In [5]:
#'LastMod_SalesUser'
columns_to_remove = ['Source','CC_Num']
report_df = report_df.drop(columns=columns_to_remove)

columns_to_process = ['PaymentsTotal','mny_GL_Currency_Payments_Amount']

# Apply the function to specified columns in the dataframe
for col in columns_to_process:
    report_df[col] = report_df[col].apply(process_fee_column)

report_df['dtm_GL_Payments_Date'] = pd.to_datetime(report_df['dtm_GL_Payments_Date'])
report_df['dtm_GL_Payments_Date'] =report_df['dtm_GL_Payments_Date'].dt.strftime('%m/%d/%Y')
report_df['dtm_GL_Payments_Date'] = pd.to_datetime(report_df['dtm_GL_Payments_Date'])

C:\Users\Ruiying.Zhang\AppData\Local\Temp\ipykernel_56148\1639469494.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  report_df['dtm_GL_Payments_Date'] = pd.to_datetime(report_df['dtm_GL_Payments_Date'])


In [7]:
#report_df.dtypes

# Compare two dataframes

In [9]:
report_accuracy,snowflake_accuracy,report_only,snowflake_only = calculate_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {snowflake_accuracy:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))
print(f"Accuracy: {report_accuracy:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 99.90% snowflake records in report
The number of records in snowflake but not in report is 17
Accuracy: 99.98% report records in snowflake
The number of records in report but not in snowflake is 3


# Output the difference file

In [10]:
try:
    snowflake_only.to_csv("Payment/"+"snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("Payment/"+"report_only_record.csv", index=False) 
except:
    print("No report only records")